In [0]:
%sql


In [0]:
-- Tabla de órdenes limpias
CREATE OR REFRESH STREAMING TABLE products.sales_silver.orders_clean_${env} (
  CONSTRAINT `order_id must be present`       EXPECT (order_id IS NOT NULL),
  CONSTRAINT `customer_id should be present`  EXPECT (customer_id IS NOT NULL),
  CONSTRAINT `valid timestamp`                EXPECT (order_timestamp IS NOT NULL) ON VIOLATION DROP ROW
)
TBLPROPERTIES ("quality" = "silver")
COMMENT "Órdenes limpias con timestamp parseado"
AS
SELECT 
  order_id,
  customer_id,
  CAST(timestamp AS TIMESTAMP) AS order_timestamp
FROM STREAM(products.sales_bronze.orders_fact_${env});

-- Tabla de ítems de orden limpios
CREATE OR REFRESH STREAMING TABLE products.sales_silver.order_items_clean_${env} (
  CONSTRAINT `order_id must be present`    EXPECT (order_id IS NOT NULL),
  CONSTRAINT `product_id should exist`     EXPECT (product_id IS NOT NULL),
  CONSTRAINT `quantity must be positive`   EXPECT (quantity > 0) ON VIOLATION DROP ROW,
  CONSTRAINT `price must be positive`      EXPECT (price > 0) ON VIOLATION DROP ROW,
  CONSTRAINT `valid timestamp`             EXPECT (item_timestamp IS NOT NULL) ON VIOLATION DROP ROW
)

TBLPROPERTIES ("quality" = "silver")
COMMENT "Ítems con cálculo de total_price (sin joins)"
AS
SELECT 
  order_id,
  product_id,
  CAST(timestamp AS TIMESTAMP) AS item_timestamp,
  quantity,
  price,
  quantity * price AS total_price
FROM STREAM(products.sales_bronze.order_items_fact_${env});
